import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd
from azureml.core.authentication import InteractiveLoginAuthentication

p = ESMLProject() # Your project settings - knows Azure services, networking, lakedesign, identity mgmt, security, etc
p.inference_mode = False

auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
#auth = InteractiveLoginAuthentication(force=True, tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)

p.active_model = 11
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.connect_to_lake()

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")

from esmlrt.interfaces.iESMLController import IESMLController
from esmlrt.interfaces.iESMLModelCompare import IESMLModelCompare
from esmlrt.interfaces.iESMLTestScoringFactory import IESMLTestScoringFactory
from esmlrt.interfaces.iESMLTrainer import IESMLTrainer
from esmlrt.runtime.ESMLController import ESMLController
from esmlrt.runtime.ESMLModelCompare2 import ESMLModelCompare
from esmlrt.runtime.ESMLTestScoringFactory2 import ESMLTestScoringFactory

In [2]:
import sys
sys.path.insert(0, "../2_A_aml_pipeline/4_inference/batch/M11/")
from your_code.your_train_code import Trainer

In [3]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

param_esml_env = "dev" 
param_inference_model_version = "1" # DATALAKE(my_model/inference/active) | settings/project_specific/active/active_scoring_in_folder.json
param_scoring_folder_date = "1000-01-01 00:00:01.243860" # DATALAKE(my_model/inference/active) | settings/project_specific/active/active_scoring_in_folder.json
param_train_in_folder_date = "1000-01-01 00:00:01.243860" # DATALAKE(my_model/train/active) | settings/project_specific/active/active_in_folder.json

p = ESMLProject(param_esml_env,param_inference_model_version,param_scoring_folder_date,param_train_in_folder_date)
#p = ESMLProject() # Alternatively use empty contructor, which takes parameters from settings\project_specific\model\active\active_in_folder.json

p.active_model = 11
p.inference_mode = False
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.verbose_logging = False
p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Environment: dev
Inference version: 1

 - ds01_diabetes
projects/project002/11_diabetes_model_reg/train/ds01_diabetes/in/dev/1000/01/01/
projects/project002/11_diabetes_model_reg/train/ds01_diabetes/out/bronze/dev/
projects/project002/11_diabetes_model_reg/train/ds01_diabetes/out/silver/dev/

 - ds02_other
projects/project002/11_diabetes_model_reg/train/ds02_other/in/dev/1000/01/01/
projects/project002/11_diabetes_model_reg/train/ds02_other/out/bronze/dev/
projects/project002/11_diabetes_model_reg/train/ds02_other/out/silver/dev/
 

Training GOLD (p.GoldPath)
projects/project002/11_diabetes_model_reg/train/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/11_diabetes_model_reg/inference/1/gold/dev/2022_10_20/264bc4d266384b13ab89cc9af085c469/
 

A)INFERENCE ONLINE: GO

In [4]:
ws = p.ws
target_column_name = p.active_model["label"]
ml_type = p.active_model["ml_type"]

esml_modelname = p.model_folder_name
esml_model_alias = p.ModelAlias
esml_current_env  = p.dev_test_prod
train_ds = p.GoldTrain
validate_ds = p.GoldValidate
test_ds = p.GoldTest
all_envs = p.get_all_envs()

secret_name_tenant = "kv-msft-weu-dev-cmnai-tenant"
secret_name_sp_id = "kv-secret-esml-project002-sp-id"
secret_name_sp_secret = "kv-secret-esml-project002-sp-secret"

test_scoring = ESMLTestScoringFactory(ml_type) # You need to implement IESMLTestScoringFactory
comparer = ESMLModelCompare(setting_path = "../../") # You need to implement IESMLModelCompare

controller = ESMLController(comparer,test_scoring,"project002",esml_modelname, esml_model_alias,all_envs, secret_name_tenant,secret_name_sp_id,secret_name_sp_secret ) # IESMLController: you do not have to change/implemen this class. Dependency injects default or your class.
controller.dev_test_prod = esml_current_env
controller.version()

...


'1.4'

## COMPARE - dev with dev

In [5]:
ws = p.ws
esml_current_env = "dev"
next_environment="dev"

##### BOTH run_ids works (since try AutoMLRun/catch Pipeline & find AutoMLRun) to find correct AutoMLRun - but get the WRONG model anyway
run_id_tag = "58be91c0-0768-4b59-bd3c-d5fbc90379ff" # = AutoMLTrainStep in main_pipeline_runid=1dcfb58a-629f-4b85-be13-795666165cc9
    ## ModelNotFound: Model with name 58be91c007684b50 not found in provided workspace

run_id_tag = "1dcfb58a-629f-4b85-be13-795666165cc9" # main_pipeline_runid=1dcfb58a-629f-4b85-be13-795666165cc9
    ## ModelNotFound: Model with name 58be91c007684b50 not found in provided workspace
#### END 


#current_model,run_id_tag, model_name = IESMLController.get_best_model_via_modeltags_only_DevTestProd(ws,controller.experiment_name)
#print("run_id_tag",run_id_tag) # 58be91c0-0768-4b59-bd3c-d5fbc90379ff = AutoMLTrainStep in main_pipeline_runid=1dcfb58a-629f-4b85-be13-795666165cc9
target_ws = controller.get_target_workspace(current_environment = esml_current_env, current_ws = ws, target_environment = esml_current_env)

#self, current_ws,current_environment, target_environment,target_workspace, experiment_name, new_model=None, new_run_id=None
promote_new_model,source_model_name,source_run_id,source_best_run,source_model,leading_model = comparer.compare_scoring_current_vs_new_model(
    new_run_id = None,#run_id_tag, #automl_step_run_id,
    new_model = None,
    current_ws = ws,
    current_environment = esml_current_env,
    target_environment = next_environment,
    target_workspace = target_ws,
    experiment_name = controller.experiment_name)

target=source environement. Compare model version in DEV/TEST/PROD with latest registered in same DEV/TEST/PROD workspace (same workspace & subscriptiom comparison)

Searching with Model list LAMBDA FILTER, on experiment_name in Model.tags called: 11_diabetes_model_reg . Meaning ESML checks for both Notebook run (AutoMLRun, Run) and PipelineRuns (AutoMLStep, PipelineRun)
E.g. Even if Pipeline experiment is called '11_diabetes_model_reg_IN_2_GOLD_TRAIN' it will be included, since original model_folder_name in ESML is '11_diabetes_model_reg' as a notebook Run experiment name. Both is included in search
DEBUG: get_best_model_via_modeltags_only_DevTestProd - get_latest_challenger ==True 11_diabetes_model_reg
TIME: FILTER ModelList:
Minutes: 4.430099324385325
DEBUG filtered_list length 33
DEBUG: model_highest_version Model(workspace=Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-R

## COMPARE - dev with test

In [5]:
ws = p.ws
esml_current_env = "dev"
next_environment="test"
target_ws = controller.get_target_workspace(current_environment = esml_current_env, current_ws = ws, target_environment = next_environment)

promote_new_model,source_model_name,source_run_id,source_best_run,source_model,leading_model = comparer.compare_scoring_current_vs_new_model(
    new_run_id = None,#run_id_tag, #automl_step_run_id,
    new_model = None,
    current_ws = ws,
    current_environment = esml_current_env,
    target_environment = next_environment,
    target_workspace = target_ws,
    experiment_name = controller.experiment_name)

Connect from DEV to TEST subscription/workspace  ( if you want to compare TEST-model with latest registered in PROD

Connecting to env: test
- ws name: msft-weu-DEV-eap-proj02_ai-amls
- self._secret_name_tenant: kv-msft-weu-dev-cmnai-tenant
- self._secret_name_project_sp_id: kv-secret-esml-project002-sp-id
- self._secret_name_project_sp_secret: kv-secret-esml-project002-sp-secret
Searching with Model list LAMBDA FILTER, on experiment_name in Model.tags called: 11_diabetes_model_reg . Meaning ESML checks for both Notebook run (AutoMLRun, Run) and PipelineRuns (AutoMLStep, PipelineRun)
E.g. Even if Pipeline experiment is called '11_diabetes_model_reg_IN_2_GOLD_TRAIN' it will be included, since original model_folder_name in ESML is '11_diabetes_model_reg' as a notebook Run experiment name. Both is included in search
TIME: FILTER ModelList:
Minutes: 4.564389169216156
- source_model.name:AutoMLd1093aff80 and version:29
- source_model.tags[status_code] esml_newly_trained 
ESML INFO:ModelComp

# Promote from DEV to TEST

In [6]:
dev_ws = p.ws
test_ws = p.get_other_workspace("test")
print(dev_ws.name)
print(test_ws.name)

msft-weu-DEV-eap-proj02_ai-amls
msft-weu-TEST-eap-proj02_ai-amls


### 1) Get best model in TEST
- 2022-10-18: 9 seconds, for 41 models. 
    - Note: This goes WAY FASTER when running as Azure ML Pipeline (since looping is cloud only, not cloud->local, cloud-local for each batch in IEnumerable)

In [7]:
current_model,run_id_tag, model_name = IESMLController.get_best_model_via_modeltags_only_DevTestProd(test_ws,controller.experiment_name)
print("Current BEST model in {} workspace is: {} from Model registry with experiment_name-TAG {}, run_id-TAG {}  model_name-TAG {}".format(test_ws.name, current_model.name,controller.experiment_name,run_id_tag,model_name))

Searching with Model list LAMBDA FILTER, on experiment_name in Model.tags called: 11_diabetes_model_reg . Meaning ESML checks for both Notebook run (AutoMLRun, Run) and PipelineRuns (AutoMLStep, PipelineRun)
E.g. Even if Pipeline experiment is called '11_diabetes_model_reg_IN_2_GOLD_TRAIN' it will be included, since original model_folder_name in ESML is '11_diabetes_model_reg' as a notebook Run experiment name. Both is included in search
TIME: FILTER ModelList:
Minutes: 0.15310513973236084
Current BEST model in msft-weu-TEST-eap-proj02_ai-amls workspace is: AutoMLd1093aff80 from Model registry with experiment_name-TAG 11_diabetes_model_reg, run_id-TAG 2a53a315-52bb-461b-a628-a733d95cf7d1  model_name-TAG AutoMLd1093aff80


In [9]:
print(current_model.tags["status_code"])
current_model

esml_promoted_2_test


Model(workspace=Workspace.create(name='msft-weu-TEST-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-TEST-RG'), name=AutoMLd1093aff80, id=AutoMLd1093aff80:2, version=2, tags={'status_code': 'esml_promoted_2_test', 'run_id': '2a53a315-52bb-461b-a628-a733d95cf7d1', 'model_name': 'AutoMLd1093aff80', 'trained_in_environment': 'dev', 'trained_in_workspace': 'msft-weu-DEV-eap-proj02_ai-amls', 'experiment_name': '11_diabetes_model_reg', 'trained_with': 'AutoMLStep', 'test_set_RMSE': '56.501912', 'test_set_R2': '0.426615', 'test_set_MAPE': '0.339886', 'test_set_Spearman_Correlation': '0.649199', 'esml_time_updated': '10/14/2022, 08:58:53', 'mflow_stage': 'Staging'}, properties={'status_code': 'esml_promoted_2_test', 'run_id': '2a53a315-52bb-461b-a628-a733d95cf7d1', 'model_name': 'AutoMLd1093aff80', 'trained_in_environment': 'dev', 'trained_in_workspace': 'msft-weu-DEV-eap-proj02_ai-amls', 'experiment_name': '11_diabetes_mod

### 2) Get best model in DEV and register to TEST
- 2022-10-18: 9 seconds, for 300 registered models. 
    - Note: This goes WAY FASTER when running as Azure ML Pipeline (since looping is cloud only, not cloud->local, cloud-local for each batch in IEnumerable)

Q: Why slow on OLD models, older than 2022-10-18?
- If not trained 2022-10-18: 4:30 LAMBDA x2
- If trained post that: 3min 54seconds is natural time, locally, if 300 models in regitry (since correct status-tag exists)
    - Note: This takes seconds not minutes, when running as Azure ML Piepeline

In [10]:
dev_ws = p.ws
dev_model,dev_run_id_tag, dev_model_name = IESMLController.get_best_model_via_modeltags_only_DevTestProd(dev_ws,controller.experiment_name)
print("Current BEST model in {} workspace is: {} from Model registry with experiment_name-TAG {}, run_id-TAG {}  model_name-TAG {}".format(dev_ws.name, dev_model.name,controller.experiment_name,dev_run_id_tag,dev_model_name))

Searching with Model list LAMBDA FILTER, on experiment_name in Model.tags called: 11_diabetes_model_reg . Meaning ESML checks for both Notebook run (AutoMLRun, Run) and PipelineRuns (AutoMLStep, PipelineRun)
E.g. Even if Pipeline experiment is called '11_diabetes_model_reg_IN_2_GOLD_TRAIN' it will be included, since original model_folder_name in ESML is '11_diabetes_model_reg' as a notebook Run experiment name. Both is included in search
TIME: FILTER ModelList:
Minutes: 4.228573095798493
Current BEST model in msft-weu-DEV-eap-proj02_ai-amls workspace is: AutoMLd1093aff80 from Model registry with experiment_name-TAG 11_diabetes_model_reg, run_id-TAG 58be91c0-0768-4b59-bd3c-d5fbc90379ff  model_name-TAG AutoMLd1093aff80


In [11]:
print(dev_model.tags["status_code"])
dev_model

esml_promoted_2_dev


Model(workspace=Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG'), name=AutoMLd1093aff80, id=AutoMLd1093aff80:30, version=30, tags={'run_id': '58be91c0-0768-4b59-bd3c-d5fbc90379ff', 'model_name': 'AutoMLd1093aff80', 'trained_in_environment': 'dev', 'trained_in_workspace': 'msft-weu-DEV-eap-proj02_ai-amls', 'experiment_name': '11_diabetes_model_reg', 'test_set_RMSE': '56.501912', 'test_set_R2': '0.426615', 'test_set_MAPE': '0.339886', 'test_set_Spearman_Correlation': '0.649199', 'status_code': 'esml_promoted_2_dev', 'mflow_stage': 'Staging'}, properties={})

In [12]:
# PROMOTE in DEV
dev_model_promoted = controller.register_model(source_ws=p.ws, target_env="dev", source_model=dev_model, esml_status=IESMLController.esml_status_promoted_2_dev)
print("DEV model promoted: {}".format(dev_model_promoted.name))
print("status_code : {}".format(dev_model.tags["status_code"]))

Connecting to env: dev
- ws name: msft-weu-DEV-eap-proj02_ai-amls
- self._secret_name_tenant: kv-msft-weu-dev-cmnai-tenant
- self._secret_name_project_sp_id: kv-secret-esml-project002-sp-id
- self._secret_name_project_sp_secret: kv-secret-esml-project002-sp-secret
Connecting to env: dev
- ws name: msft-weu-DEV-eap-proj02_ai-amls
- self._secret_name_tenant: kv-msft-weu-dev-cmnai-tenant
- self._secret_name_project_sp_id: kv-secret-esml-project002-sp-id
- self._secret_name_project_sp_secret: kv-secret-esml-project002-sp-secret
target=source environement. Compare model version in DEV/TEST/PROD with latest registered in same DEV/TEST/PROD workspace (same workspace & subscriptiom comparison)

Register in workspace: msft-weu-DEV-eap-proj02_ai-amls
Registering model AutoMLd1093aff80
DEV model promoted: {} AutoMLd1093aff80
status_code : esml_promoted_2_dev


In [13]:
if ("esml_time_updated" in dev_model.tags):
    print("esml_time_updated: {}".format(dev_model.tags["esml_time_updated"]))
print("status_code : {}".format(dev_model.tags["status_code"]))
print("model_name  : {}".format(dev_model.tags["model_name"]))
print("trained_in_workspace   : {}".format(dev_model.tags["trained_in_workspace"]))

status_code : esml_promoted_2_dev
model_name  : AutoMLd1093aff80
trained_in_workspace   : msft-weu-DEV-eap-proj02_ai-amls


In [14]:
# REGISTER dev model to TEST
dev2test_model_registered = controller.register_model(source_ws=p.ws, target_env="test", source_model=dev_model)
print("dev2test_model_registered: {}", dev2test_model_registered.name)

Connecting to env: test
- ws name: msft-weu-DEV-eap-proj02_ai-amls
- self._secret_name_tenant: kv-msft-weu-dev-cmnai-tenant
- self._secret_name_project_sp_id: kv-secret-esml-project002-sp-id
- self._secret_name_project_sp_secret: kv-secret-esml-project002-sp-secret
Connecting to env: dev
- ws name: msft-weu-DEV-eap-proj02_ai-amls
- self._secret_name_tenant: kv-msft-weu-dev-cmnai-tenant
- self._secret_name_project_sp_id: kv-secret-esml-project002-sp-id
- self._secret_name_project_sp_secret: kv-secret-esml-project002-sp-secret
Connect from DEV to TEST subscription/workspace  ( if you want to compare TEST-model with latest registered in PROD

Connecting to env: test
- ws name: msft-weu-DEV-eap-proj02_ai-amls
- self._secret_name_tenant: kv-msft-weu-dev-cmnai-tenant
- self._secret_name_project_sp_id: kv-secret-esml-project002-sp-id
- self._secret_name_project_sp_secret: kv-secret-esml-project002-sp-secret
Register in workspace: msft-weu-TEST-eap-proj02_ai-amls
Registering model AutoMLd1093a

### 3) Verify - that the BEST model in TEST (date registered) now is different from the one in step 1 check

In [15]:
test_model_again,run_id_tag, model_name = IESMLController.get_best_model_via_modeltags_only_DevTestProd(test_ws,controller.experiment_name)
print("Current BEST model in {} workspace is: {} from Model registry with experiment_name-TAG {}, run_id-TAG {}  model_name-TAG {}".format(test_ws.name, test_model_again.name,controller.experiment_name,run_id_tag,model_name))
print("")
if ("esml_time_updated" in test_model_again.tags):
    print("esml_time_updated: {}".format(test_model_again.tags["esml_time_updated"]))
print("status_code : {}".format(test_model_again.tags["status_code"]))
print("model_name  : {}".format(test_model_again.tags["model_name"]))
print("trained_in_workspace   : {}".format(test_model_again.tags["trained_in_workspace"]))

Searching with Model list LAMBDA FILTER, on experiment_name in Model.tags called: 11_diabetes_model_reg . Meaning ESML checks for both Notebook run (AutoMLRun, Run) and PipelineRuns (AutoMLStep, PipelineRun)
E.g. Even if Pipeline experiment is called '11_diabetes_model_reg_IN_2_GOLD_TRAIN' it will be included, since original model_folder_name in ESML is '11_diabetes_model_reg' as a notebook Run experiment name. Both is included in search
TIME: FILTER ModelList:
Minutes: 0.14623868068059284
Current BEST model in msft-weu-TEST-eap-proj02_ai-amls workspace is: AutoMLd1093aff80 from Model registry with experiment_name-TAG 11_diabetes_model_reg, run_id-TAG 58be91c0-0768-4b59-bd3c-d5fbc90379ff  model_name-TAG AutoMLd1093aff80

status_code : esml_promoted_2_test
model_name  : AutoMLd1093aff80
trained_in_workspace   : msft-weu-DEV-eap-proj02_ai-amls


## 1,2,3 all together

In [ ]:
p.dev_test_prod = "dev" # TODO: change to test when prod is up
controller.dev_test_prod = "test" # TODO: change to test when prod is up
next_environment = controller.get_next_environment()
model = test_model_again

next_environment = controller.get_next_environment() # Test, or PROD
promote_new_model,source_model_name,source_run_id,source_best_run,source_model,target_model = comparer.compare_scoring_current_vs_new_model(
    new_run_id = run_id_tag, #main_run.id,
    current_ws = p.ws,
    current_environment = "dev",
    target_environment = "test",
    target_workspace = test_ws,
    experiment_name = controller.experiment_name)

print("Compared 2nd time - Outer loop")
if (promote_new_model == True):
    model_registered_in_target = controller.register_model(source_ws=p.ws, target_env=next_environment, source_model=model,esml_status=IESMLController.esml_status_promoted_2_test)
    print("Registered model {} with version {} in TEST".format(model_registered_in_target.name,model_registered_in_target.version))

# VARIOUS things

In [ ]:
controller.experiment_name

In [ ]:
from azureml.train.automl.run import AutoMLRun

automl_step_run_id = '8b7c996f-6937-4470-a42b-7e59d91f6d49'

experiment_run = ws.experiments[controller.experiment_name] # Get the experiment. Alternatively: Experiment(workspace=source_workspace, name=experiment_name)
automl_step_run = AutoMLRun(experiment_run, run_id = automl_step_run_id)
best_run, fitted_model_1 = automl_step_run.get_output()

promote_new_model,source_model_name,source_run_id,source_best_run,source_model,target_model = comparer.compare_scoring_current_vs_new_model(
    new_run_id = automl_step_run_id, # main_run.id,
    current_ws = ws,
    current_environment = esml_current_env,
    target_environment = esml_current_env,
    target_workspace = ws,
    experiment_name = controller.experiment_name)

In [ ]:
print(IESMLController.esml_status_new)

In [ ]:
controller.dev_test_prod

# Get BEST MODEL

In [ ]:
experiment, model,main_run, best_automl_run,fitted_model = controller.get_latest_run_via_PipelineMetaDataset(ws,controller.experiment_name, controller.dataset_gold_train_runinfo_name_azure)
print("     ###    ")
print("Experiment:", experiment.name)
print("Pipeline RUN:", main_run.id)
print("Best RUN:",best_automl_run)
print ("...and...")

print("AML Model()", type(model))
print("Fitted model", type(fitted_model))

In [ ]:
main_run

In [ ]:
experiment, model,main_run, best_run,fitted_model = IESMLController.get_best_model_run_fitted_model_Dev(ws,controller.experiment_name, get_latest_challenger=False)
print(experiment.name)
print(model.name)
print(main_run.id)
try:
    print(model.tags['esml_time_updated'])
except:
    print("Not an ESML registered model..")
    
print ("...and...")
print(type(best_run))
print(type(fitted_model))

In [ ]:
model_highest_version, run_id_tag, model_name_tag = IESMLController.get_best_model_via_modeltags_only_DevTestProd(ws,controller.experiment_name,get_latest_challenger=False,filter_on_version=None,sort_by_created_instead_of_version=False) # 3:39 LAMBDA
print(model_highest_version.name)
print(run_id_tag)
print(model_name_tag)
# 4.24 Datetime sort
try:
    print(model_highest_version.version) # AutoML00b58dfed0 (v17) (manual run) VS 11_diabetes_model_reg(v 11) -> Both are TAGGED with experiment_name=11_diabetes_model_reg
    print(model_highest_version.created_time)
    print(model_highest_version.tags['esml_time_updated'])
except:
    print("Not an ESML registered model..")

print("model_highest_version.run is None: {}".format((model_highest_version.run is None)))

In [ ]:
from azureml.core import Model
all_models = Model.list(workspace=ws)
filtered_list = list(filter(lambda r: (r.tags.get("experiment_name") == controller.experiment_name), all_models))

filtered_list.sort(key=lambda r: r.version,reverse=True)
            
if (len(filtered_list) > 0): # IF we found any...
    model_highest_version = filtered_list[0]

print(model_highest_version.version)
print(model_highest_version.created_time)

In [ ]:
print(model_highest_version.created_time)

# get TEST SCORING

In [ ]:
#  label,ws, GoldTest, model, fitted_model, source_best_run/run
experiment, model,main_run, best_automl_run,fitted_model = IESMLController.get_best_model_run_fitted_model_Dev(ws,controller.experiment_name)

In [ ]:
best_automl_run.properties['model_name']

In [ ]:
type(fitted_model)

In [ ]:
model, rmse, r2, mean_abs_percent_error,mae,spearman_correlation,plt, dummy = test_scoring.get_test_scoring_8(ws,target_column_name,test_ds,fitted_model,best_automl_run,model)

# REGISTER model in DEV

In [ ]:
experiment, model2,main_run, best_automl_run,fitted_model = IESMLController.get_best_model_run_fitted_model_Dev(ws,controller.experiment_name) # Get model again

In [ ]:
model_registered_in_target = controller.register_model(source_ws=ws, target_env="dev", source_model_to_copy_tags_from=model2,esml_status=IESMLController.esml_status_promoted_2_dev)

# REGISTER model from DEV to TEST

In [ ]:
model_registered_in_target = controller.register_model(source_ws=ws, target_env="test", source_model_to_copy_tags_from=model2)

# End-2-End

In [ ]:
experiment, model,main_run, best_automl_run,fitted_model = IESMLController.get_best_model_run_fitted_model_Dev(ws,controller.experiment_name)

In [ ]:
esml_modelname

In [ ]:
def train_test_compare_register(ws,target_column_name,esml_modelname,esml_model_alias, esml_current_env, train_ds,validate_ds,test_ds):
    test_scoring = None # IESMLTestScoringFactory
    comparer = None # IESMLModelCompare
    trainer = None # IESMLTrainer

    # CUSTOMIZE ############### Optional: You can CUSTOMIZE, by implementing your own CLASS that supports interfaces/abstract classes: IESMLTestScoringFactory,IESMLModelCompare
    ml_type = "regression"
    test_scoring = ESMLTestScoringFactory(ml_type) # You need to implement IESMLTestScoringFactory
    comparer = ESMLModelCompare(setting_path = "") # You need to implement IESMLModelCompare
    # CUSTOMIZE END ###############

    secret_name_tenant = "kv-msft-weu-dev-cmnai-tenant"
    secret_name_sp_id = "kv-secret-esml-project002-sp-id"
    secret_name_sp_secret = "kv-secret-esml-project002-sp-secret"

    controller = ESMLController(comparer,test_scoring,"project002",esml_modelname, esml_model_alias, secret_name_tenant,secret_name_sp_id,secret_name_sp_secret) # IESMLController: you do not have to change/implemen this class. Dependency injects default or your class.
    controller.dev_test_prod = esml_current_env

    ##1 ) Get "current" 'last_gold_training_run' [pipeline_run_id, training_data_used]

    experiment, model,main_run, best_automl_run,fitted_model_1 = IESMLController.get_best_model_run_fitted_model_Dev(ws,controller.experiment_name)

    # ITrainer: Defaults to using AutoML. Optionally you can implement this. Else you need to implement ITrainer in 'YourTrainer' class
    trainer = Trainer(model.name,esml_modelname,esml_model_alias, esml_current_env, ml_type,train_ds,validate_ds,test_ds)
    train_run, aml_model,fitted_model_2 = trainer.train(train_ds,validate_ds)
    
    #  label,ws, GoldTest, model, fitted_model, source_best_run/run
    model, rmse, r2, mean_abs_percent_error,mae,spearman_correlation,plt, dummy = test_scoring.get_test_scoring_8(ws,target_column_name,test_ds,fitted_model_1,best_automl_run,model)

    next_environment = controller.get_next_environment()

    #  current_ws,current_environment, target_environment,target_workspace, experiment_name)
    target_ws = controller.get_target_workspace(current_environment = esml_current_env, current_ws = ws, target_environment = esml_current_env)

    ## 2) COMPARE if better

    promote_new_model,source_model_name,source_run_id,source_best_run,source_model,target_model = comparer.compare_scoring_current_vs_new_model(
        new_run_id = main_run.id,
        current_ws = ws,
        current_environment = esml_current_env,
        target_environment = esml_current_env,
        target_workspace = target_ws,
        experiment_name = trainer.experiment_name)


    print("compared once, 1 time, inner loop - Dev")

    ## 3) REGISTER model, if better

    if (promote_new_model == True): # Better than all in DEV?! (Dev or Test,  is usually current_env)
        model_registered_in_target = controller.register_model(source_ws=ws, target_env=esml_current_env, source_model_to_copy_tags_from=model)
        print("registered in DEV")

        # Better than all in DEV, Lets check if its better than all in TEST? (or prod)
        next_environment = controller.get_next_environment() # Test, or PROD
        promote_new_model,source_model_name,source_run_id,source_best_run,source_model,target_model = comparer.compare_scoring_current_vs_new_model(
            new_run_id = main_run.id,
            current_ws = ws,
            current_environment = esml_current_env,
            target_environment = next_environment,
            target_workspace = target_ws,
            experiment_name = trainer.experiment_name)

        print("Compared 2nd time - Outer loop")
        if (promote_new_model == True):
            model_registered_in_target = controller.register_model(source_ws=ws, target_env="test", source_model_to_copy_tags_from=model)
            print("Registered in TEST")

train_test_compare_register(ws,target_column_name,esml_modelname,esml_model_alias, esml_current_env, train_ds,validate_ds,test_ds)

# DATASET: See Run info from last gold scored

In [ ]:
from azureml.core.dataset import Dataset
# p = ESMLProject()
# ...
# ...p.connect_to_lake()
meta_dataset = Dataset.get_by_name(p.ws, name="M10_GOLD_SCORED_RUNINFO")
meta_dataset.to_pandas_dataframe()